In [ ]:
import os
from glob import glob
import numpy as np

train_data_path = 'dataset/train/'
test_data_path = 'dataset/test/'

In [ ]:
def load_folder(path):
    pattern = os.path.join(path, '*.pose.npy')
    pose_paths = glob(pattern)
    result = []
    for pose_path in pose_paths:
        face_path = pose_path.replace('pose', 'face')
        try:
            pose = np.load(pose_path)[0]#[[0, 1, 2, 5, 15, 16, 17, 18]]
            pose = pose.flatten()
            face = np.load(face_path)[0].flatten()
            joined = np.concatenate([pose, face])
            result.append(joined)
        except:
            print(pose_path, 'seems to contain bad data:', np.load(pose_path))
    prev_shape = None
    for x in result:
        prev_shape = x.shape
    return np.stack(result, axis=0)
              
def load_dataset(data_path):
    good_examples_path = os.path.join(data_path, 'Good')
    bad_examples_path = os.path.join(data_path, 'Bad')
    good = load_folder(good_examples_path)
    good_Y = np.ones(good.shape[0])
    bad = load_folder(bad_examples_path)
    bad_Y = np.zeros(bad.shape[0])
    X = np.concatenate([good, bad], axis=0)
    Y = np.concatenate([good_Y, bad_Y], axis=0)
    return X, Y
    
X, Y = load_dataset(train_data_path)
print(X.shape, Y.shape)
X_test, Y_test = load_dataset(test_data_path)
print(X_test.shape, Y_test.shape)

In [ ]:
mean = X.mean(axis=0)
std = X.std(axis=0)
std[std < 1e-5] = 1
X -= mean
X /= std
X_test -= mean
X_test /= std

In [ ]:
import sklearn
import sklearn.linear_model
import sklearn.svm

#model = sklearn.linear_model.LogisticRegression(max_iter=1000)
model = sklearn.svm.SVC(probability=True)

In [ ]:
model.fit(X, Y)

In [ ]:
Y_pred = model.predict(X_test)

In [ ]:
import sklearn.metrics
sklearn.metrics.accuracy_score(Y_pred, Y_test)

In [ ]:
Y_test.sum() / Y_test.shape[0]

In [ ]:
def get_roc_auc(model, x_test, y_test):
    y_pred = model.predict_proba(x_test)[:, 1]
    roc = sklearn.metrics.roc_curve(y_test, y_pred)
    auc = sklearn.metrics.auc(roc[0], roc[1])
    return roc, auc

In [ ]:
roc, auc = get_roc_auc(model, X_test, Y_test)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(roc[0], roc[1],
         lw=2, label='ROC curve (area = %0.2f)' % (auc))
plt.legend()